In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_excel('data_full_5sekres.xlsx')

In [ ]:
df1 = df.copy()

In [ ]:
#Create a column of load profile
df1['Load'] = df1['thrusterAndPropulsion']
df1['BaseLoad'] = 300
df1.loc[df1['operationId'] == 6, 'BaseLoad'] = 0
df1.loc[df1['operationId'] == 1, 'BaseLoad'] = 0
df1['Load'] = df1['thrusterAndPropulsion'] + df1['BaseLoad']

df1['Battery'] = 0
df1['SOC'] = 0
#Create dataset with only relevant columns
df3 = df1[['datetime', 'Load', 'engMdoPower_0_','engMdoPower_1_','engMdoPower_2_', 'engMdoPower_3_', 'operationId', 'BaseLoad', 'Battery', 'SOC']]

#Set engine powers to zero
num = df3['engMdoPower_0_']._get_numeric_data()
num[num != 0] = 0

num = df3['engMdoPower_1_']._get_numeric_data()
num[num != 0] = 0

num = df3['engMdoPower_2_']._get_numeric_data()
num[num != 0] = 0

num = df3['engMdoPower_3_']._get_numeric_data()
num[num != 0] = 0


display(df3)

In [ ]:
df5 = df3.copy()

load = df5['Load'].values
dg0 = df5['engMdoPower_0_'].values
dg1 = df5['engMdoPower_1_'].values
dg2 = df5['engMdoPower_2_'].values
dg3 = df5['engMdoPower_3_'].values
bat = df5['Battery'].values
soc = [0]*120960
soc[0] = 124 #124 empty (20%)
opid = df5['operationId'].values


for i in range(len(load)):
    if i == 120959:
        pass
    elif load[i] <= 2100 and load[i+1] <= 2100: 
        if opid[i] == 100 and soc[i] <= 124: #DP MODE, empty battery, two generators
            bat[i] = -100
            soc[i+1] = soc[i] - bat[i]/720
            dg0[i] = dg1[i] = (load[i] - bat[i])/2
        elif opid[i] == 100 and soc[i] > 124: #DP MODE, battery spinning reserve
            bat[i] = 0
            soc[i+1] = soc[i]
            dg0 = load[i]
        elif opid[i] == 3 and soc[i] > 124:#Transit
            bat[i] = 0
            soc[i+1] = soc[i]
            dg0[i] = load[i]
        elif opid[i] == 5:
        elif opid[i] == 7: #standby
            bat[i] = 0
            soc[i+1] = soc[i]
            dg0[i] = load[i]            
        elif opid[i] == 6: #Port shore connection
            bat[i] = 0
            soc[i+1] = soc[i]
            dg0[i] = 0             
        elif opid[i] == 1: #Port normal
            bat[i] = 0
            soc[i+1] = soc[i]
            dg0[i] = load[i] 
    elif load[i] <= 2100 and load[i+1] > 2100
    
    elif load[i] <= 4200: #DP mode low load, single generator and battery as spinning reserve
        if opid[i] == 100:
            bat[i] = 0
            soc[i+1] = soc[i]
            dg0[i] = load[i]
        elif opid[i] == 3 or opid[i] == 5: #Transit
            bat[i] = 0
            soc[i+1] = soc[i]
            dg0[i] = load[i]
        elif opid[i] == 7: #standby
            bat[i] = 0
            soc[i+1] = soc[i]
            dg0[i] = load[i]            
        elif opid[i] == 6: #Port shore connection
            bat[i] = 0
            soc[i+1] = soc[i]
            dg0[i] = 0             
        elif opid[i] == 1: #Port normal
            bat[i] = 0
            soc[i+1] = soc[i]
            dg0[i] = load[i]         
      